In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingRegressor

In [3]:
data = pd.read_csv('CC2020_train_final.csv')

In [4]:
data.head(3)

,YEAR,LOCATION,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER,YIELD
0,18,Loc 5608,Inbred_142,Cluster11,Tester_740,Cluster10,1.135462
1,18,Loc 4533,Inbred_142,Cluster11,Tester_740,Cluster10,1.139813
2,18,Loc 5620,Inbred_142,Cluster11,Tester_740,Cluster10,1.117778


In [5]:
X = data.drop(['YEAR', 'LOCATION'], axis=1)

In [6]:
len(X)

199476

In [7]:
clean_data = X.groupby(['INBRED', 'INBRED_CLUSTER','TESTER', 'TESTER_CLUSTER' ],as_index=False).mean()

In [8]:
clean_data.head(5)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER,YIELD
0,Inbred_1071,Cluster8,Tester_1345,Cluster1,0.986544
1,Inbred_1071,Cluster8,Tester_4373,Cluster3,1.057704
2,Inbred_1071,Cluster8,Tester_4473,Cluster5,1.023704
3,Inbred_1071,Cluster8,Tester_4541,Cluster4,1.014735
4,Inbred_1071,Cluster8,Tester_5305,Cluster4,1.062727


In [9]:
len(clean_data)

10919

In [10]:
y = clean_data.YIELD

In [11]:
data = clean_data.drop(['YIELD'], axis=1)

In [12]:
data.head(5)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER
0,Inbred_1071,Cluster8,Tester_1345,Cluster1
1,Inbred_1071,Cluster8,Tester_4373,Cluster3
2,Inbred_1071,Cluster8,Tester_4473,Cluster5
3,Inbred_1071,Cluster8,Tester_4541,Cluster4
4,Inbred_1071,Cluster8,Tester_5305,Cluster4


In [13]:
X = data

In [14]:
label_enc = preprocessing.LabelEncoder()

In [15]:
X['INBRED'] = label_enc.fit_transform(X['INBRED'])
X['INBRED_CLUSTER'] = label_enc.fit_transform(X['INBRED_CLUSTER'])
X['TESTER'] = label_enc.fit_transform(X['TESTER'])
X['TESTER_CLUSTER'] = label_enc.fit_transform(X['TESTER_CLUSTER'])

In [16]:
X.head(3)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER
0,0,12,0,0
1,0,12,125,7
2,0,12,136,9


In [17]:
#notice how these are all same.. number varies within same location or year

In [18]:
enc = preprocessing.OneHotEncoder()

In [19]:
X = enc.fit_transform(X)

#Classifiers

In [20]:
#GradientBoosting Basic

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=np.random)

In [21]:
lr = make_pipeline(LinearRegression())
rfr = make_pipeline(RandomForestRegressor())
gbr = make_pipeline(GradientBoostingRegressor())

In [22]:
estimators = [ ('lr', LinearRegression(n_jobs = -1)), ('rfr', RandomForestRegressor(n_estimators= 1000, min_samples_split=12,min_samples_leaf = 1, max_features='sqrt', 
                                     max_depth =130, bootstrap=True, n_jobs = -1))]

In [23]:
reg = StackingRegressor( estimators=estimators, final_estimator=GradientBoostingRegressor(subsample = 0.7, n_estimators=800, learning_rate=0.05,
                                   max_depth=9, max_features='sqrt',
                                   min_samples_leaf=2, min_samples_split=2, 
                                   loss='ls', random_state =5))

In [26]:
reg.fit(X_train, y_train)

StackingRegressor(cv=None,
                  estimators=[('lr',
                               LinearRegression(copy_X=True, fit_intercept=True,
                                                n_jobs=-1, normalize=False)),
                              ('rfr',
                               RandomForestRegressor(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     criterion='mse',
                                                     max_depth=130,
                                                     max_features='sqrt',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
              

In [27]:
predictions = reg.predict(X_test)

In [28]:
mean_absolute_error(predictions, y_test)

0.04390136923643336